In [2]:
%pylab inline
import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('ticks')
sns.set_context('paper')
import sys

sys.path.append('/Users/wayment/software')
import pynmrstar

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv('BMRB_T1T2NOE_set_aug92024update_MANUALLY_CURATED.csv')

# last update Dec 9, 2024

subset = [4267,4390,4689,4762,5154,5330,5518,5687,5762,5839,5841,5991,5995,6470,6758,6838,7035,7036,7056,7432,11080,15255,15445,15451,15521,15795,16234,
          16392,16426,16737,17018,17246,17266,17306,17701,17783,17881,18092,18257,18260,18306,18380,18477,18758,18772,18773,18903,19127,19153,19189,19335,19356,
          25013,25636,26513,26723,27011,27194,27888,30523,34545,36171,50020,50233,50332,50410,50734,51087,51230,51416]
df = df.loc[df.entry_ID.isin(subset)]

seq_diff={18257: 9, 6758: 8, 15445: 8, 15975: 10, 16392: 23, 16737: 6, 17246: 6, 17306: 6, 17701: 6, 17783: 10, 18092: 6, 18477: 8, 27011: 6, 50410: 7,18903: 81}
N_or_C={18257: 'N', 6758: 'N', 15445: 'N', 15975: 'N', 16392: 'N', 16737: 'C', 17246: 'C', 17306: 'C', 17701: 'C', 17783: 'N', 18092: 'C', 18477: 'N', 27011: 'C', 50410: 'C',18903: 'C'}


In [4]:
len(subset) # Dec 9: 71. Jan 20: 70, removed 28132

70

In [5]:
def convert_units(val, err, unit, pseudo=1e-8):
    '''Takes in values and value errors (T1 or T2)
    and standardizes them to be rates with units of s-1
    
    Inputs:
    val: list-like, values of T1 or T2 or Rex
    err: list-like, values of associated error
    unit: str from BMRB entry (s, s-1, Hz, ms-1, ms)
    pseudo: pseudocount to avoid dividing by zero
    
    Outputs:
    out: list-like, normalized rate in (s-1) units
    out_err, list-like, normalized error on rate in (s-1)
    '''
    
    if unit=='s':
        out = 1/val
        out_err = out * err/val
        return out, out_err
    
    elif unit == 's-1' or unit == 'Hz':
        return val, err

    elif unit=='ms-1':
        return 1000*val, 1000*err
    
    elif unit== 'ms':
        out = 1/val
        out_err = out*err/val
        return 1000*out, 1000*out_err
    else:
        raise runtimeError('Unit type not understood.')


def get_data(row, expt_kw='T1_experiment', val_kw='Val', err_kw='Val_err', unit_kw='T1_val_units'):
    if pd.isna(row[expt_kw]):
        return np.zeros(len(row['sequence']))*np.NaN,np.zeros(len(row['sequence']))*np.NaN 
    entry = pynmrstar.Entry.from_database(int(row['entry_ID']))
    
#     if row['entry_ID'] == 19153:
#         entry.rename_saveframe('heteronuclear_list_T1_2','heteronuclear_T1_list_2')

    loops = entry[row[expt_kw]]._loops
    
    #Find correct loop
    for i, loop in enumerate(loops):
        if val_kw in loop._tags:
            loop_ind = i
            break
    loop = loops[loop_ind]
    
    # initialize arrays
    #seq = entry.get_saveframes_by_category('entity')[0]['Polymer_seq_one_letter_code'][0].replace('\n','')
    seq = row['sequence']
    T, T_err = np.ones(len(seq))*np.nan, np.ones(len(seq))*np.nan
    
#    atom_id_lst=[]

    # Loop thru data
    for i, res in enumerate(loop.data):
        T_val = res[loop._tags.index(val_kw)]
        T_val_err = res[loop._tags.index(err_kw)]
        atom_id = res[loop._tags.index('Atom_ID')] # keeping track of this to remove second peaks for Arg, Trp. Only happens in entry 4267
        # if atom_id not in atom_id_lst:
        #     atom_id_lst.append(atom_id)
        seqpos = int(res[loop._tags.index('Seq_ID')]) - 1 # Zero-indexing seqpos here.
        #resname = seq1(res[loop._tags.index('Comp_ID')])

        if T_val != '.' and np.isnan(T[seqpos]) and atom_id in ['N','H']: # no value yet, doing this to prevent second sidechain atom
            if float(T_val) != 0:
                T[seqpos] = float(T_val)
                if T_val_err != '.':
                    T_err[seqpos] = float(T_val_err)
        else:
            continue
            
    # convert units
    if unit_kw is not None:
        unit = entry[row[expt_kw]][unit_kw][0]
        if unit=='.':
            unit = entry[row[expt_kw]]['T2_val_units'][0]
        if row['entry_ID']==5154: unit='s'
        if row['entry_ID']==7056: unit='s'
        if row['entry_ID']==7088: unit='s'
        if row['entry_ID']==15930: unit='s'
        if row['entry_ID']==16737: unit='ms'
        if row['entry_ID']==17701: unit='ms'
        if row['entry_ID']==17266: unit='s-1'
        if row['entry_ID']==18087: unit='s-1'
        if row['entry_ID']==18758: unit='s-1'
        if row['entry_ID']==50734: unit='ms'
        if row['entry_ID']==50745: unit='ms'
        if row['entry_ID']==51306 and val_kw=='Val': unit='s'
        if row['entry_ID']==51306 and val_kw=='T2_val': unit='s-1'
        if row['entry_ID']==51478: unit='s-1'
        if row['entry_ID']==51126: unit='s-1'
        
    # 26788 T2 switched column of error and value
    # 17266 also switched column of T2 error and value, and stated unit is s for it when it should be s-1
    if row['entry_ID'] == 26788 and expt_kw=='T2_experiment':
        T, T_err = convert_units(T_err, T, unit)
    elif row['entry_ID'] == 17266 and expt_kw=='T2_experiment':
        T, T_err = convert_units(T_err, T, unit)
    else:
        T, T_err = convert_units(T, T_err, unit)
        
    #27011 flipped error, confirmed this with authors
    if row['entry_ID'] ==27011 and expt_kw=='T2_experiment':
        T_err = 1/T_err
        
    # 15486: they deposited T1/T2 in value column and R1/R2 in err column
    if row['entry_ID'] == 15486: #or row['entry_ID'] == 17266:
        T_err = np.zeros(len(row['sequence']))*np.NaN

    # if row['entry_ID']==7036: # deposited seq was different than data seqpos. Fixed in manual csv, should start with "HVVQ..."
    #     T = T[:-2]
    #     T_err = T_err[:-2]
            
    return T, T_err

def get_NOE_data(row, expt_kw='NOE_experiment', val_kw='Val', err_kw='Val_err'):
    if pd.isna(row[expt_kw]):
        return np.zeros(len(row['sequence']))*np.NaN,np.zeros(len(row['sequence']))*np.NaN 
    entry = pynmrstar.Entry.from_database(row['entry_ID'])
    
#     if row['entry_ID'] == 19153:
#         entry.rename_saveframe('heteronuclear_list_T1_2','heteronuclear_T1_list_2')

    loops = entry[row[expt_kw]]._loops
    
    #Find correct loop
    for i, loop in enumerate(loops):
        if val_kw in loop._tags:
            loop_ind = i
            break
    loop = loops[loop_ind]
    
    # initialize arrays
    #seq = entry.get_saveframes_by_category('entity')[0]['Polymer_seq_one_letter_code'][0].replace('\n','')
    seq = row['sequence']
    T, T_err = np.ones(len(seq))*np.nan, np.ones(len(seq))*np.nan

    # Loop thru data
    for i, res in enumerate(loop.data):
        T_val = res[loop._tags.index(val_kw)]
        T_val_err = res[loop._tags.index(err_kw)]
        atom_id = res[loop._tags.index('Atom_ID_1')] # keeping track of this to remove second peaks for Arg, Trp. Only happens in entry 4267
        seqpos = int(res[loop._tags.index('Seq_ID_1')]) - 1 # Zero-indexing seqpos here.
        #resname = seq1(res[loop._tags.index('Comp_ID')])
        
        atom_pass=False
        if atom_id in ['N','H'] or row['entry_ID'] in [6470, 5330, 5687, 5720]: # these didn't put anything for atom_id
            atom_pass=True

        if T_val != '.' and np.isnan(T[seqpos]) and atom_pass: # no value yet, doing this to prevent second sidechain atom
            if float(T_val) != 0:
                T[seqpos] = float(T_val)
                if T_val_err != '.':
                    T_err[seqpos] = float(T_val_err)
        else:
            continue
            
    if row['entry_ID']==18257: # values are reported as relative intensities, not normalized to reference integral
        T = np.zeros(214)*np.NaN
        T_err = np.zeros(214)*np.NaN
        
    if row['entry_ID']==19356: 
        T /= np.nanmax(T)
        T_err = np.zeros(88)*np.NaN
        
    if row['entry_ID']==16737: 
        T /= np.nanmax(T)   
        
    return T, T_err

df[['R1','R1_err']] = df.apply(lambda row: get_data(row), axis=1, result_type='expand')
df[['R2','R2_err']] = df.apply(lambda row: get_data(row,expt_kw='T2_experiment', val_kw='T2_val', err_kw='T2_val_err', unit_kw = 'T2_val_units'), axis=1, result_type='expand')
df[['NOE','NOE_err']] = df.apply(lambda row: get_NOE_data(row), axis=1, result_type='expand')

In [6]:
def adjust(row, MET):
    if row['entry_ID'] in seq_diff.keys():
        offset=seq_diff[row['entry_ID']]
        
        if N_or_C[row['entry_ID']]=='N':
            return row[MET][offset:], row[MET+'_err'][offset:]
        else: 
            return row[MET][:-1*offset], row[MET+'_err'][:-1*offset]
    else:
        return row[MET], row[MET+'_err']

df[['R1','R1_err']] = df.apply(lambda row: adjust(row, 'R1'), axis=1, result_type='expand')
df[['R2','R2_err']] = df.apply(lambda row: adjust(row, 'R2'), axis=1, result_type='expand')
df[['NOE','NOE_err']] = df.apply(lambda row: adjust(row, 'NOE'), axis=1, result_type='expand')

In [7]:
for ind in [7035, 7036, 18257]:
    print(ind)
    entry = df.loc[df.entry_ID==ind].iloc[0]
    noe = pd.read_csv(f'patched_NOEs/{ind}_NOE.csv')

    noe_vec = np.zeros(len(entry['sequence']))*np.NaN
    noe_err_vec = np.zeros(len(entry['sequence']))*np.NaN

    for _, row in noe.iterrows():
        noe_vec[int(row['seqpos']-1)] = row['NOE']
        noe_err_vec[int(row['seqpos']-1)] = row['NOE_err']

    df.at[entry.name,'NOE'] = noe_vec
    df.at[entry.name,'NOE_err'] = noe_err_vec
    
    df.at[entry.name,'NOE_expt_index']='manual'
    
df.at[entry.name,'NOE'] = df.at[entry.name,'NOE'][:-9]
df.at[entry.name,'NOE_err'] = df.at[entry.name,'NOE_err'][:-9]

7035
7036
18257


In [8]:
def find_any_missing(row):
    n_missing_expts = 0
    if row['T1_expt_index'] == -1:
        n_missing_expts +=1
    if row['T2_expt_index'] == -1:
        n_missing_expts +=1
    if row['NOE_expt_index'] == -1 :
        n_missing_expts +=1
        
    return n_missing_expts

#Remove any fields that don't have all 3 expts

df['n_missing_expts'] = df.apply(lambda row: find_any_missing(row), axis=1)
print(df.groupby('n_missing_expts').size())
df['seq_len'] = [len(x) for x in df['sequence']]

df = df.loc[df.n_missing_expts==0]

#Correct 0 and NaN uncertainties

def fix_errs(row, dat_type):
    multiplier={'R1': 0.1, 'R2': 0.05,'NOE': 0.05}
    dat = row[dat_type+'_err'].copy()
    
    if np.isnan(dat).all():
        print('updating nans in', row['entry_ID'], row['field_strength'], dat_type)
        return multiplier[dat_type] * np.abs(row[dat_type])
    
    elif np.any(dat==0):
        print('fixing zero in', row['entry_ID'], row['field_strength'])
        nonzeros = dat[np.where(dat>0)]
        dat[dat==0] = np.min(nonzeros)
        return np.abs(dat)
    else:
        return np.abs(row[dat_type+'_err'])

for dat_type in ['R1','R2','NOE']:
    df[dat_type+'_err'] = df.apply(lambda row: fix_errs(row, dat_type), axis=1)

df.to_json('BMRB_subset_RelaxDB_20jan2025.json.zip')

n_missing_expts
0    90
1     2
2     2
dtype: int64
updating nans in 6470 600 R1
fixing zero in 5839 600
updating nans in 7035 800 R1
updating nans in 7036 800 R1
updating nans in 17018 500 R1
fixing zero in 18260 600
fixing zero in 19335 600
fixing zero in 51230 750
updating nans in 4689 500 R2
updating nans in 6470 600 R2
updating nans in 7035 800 R2
updating nans in 7036 800 R2
updating nans in 17018 500 R2
fixing zero in 17246 500
fixing zero in 18260 600
fixing zero in 51087 850
updating nans in 4689 500 NOE
updating nans in 6470 600 NOE
fixing zero in 6758 600
updating nans in 17018 500 NOE
fixing zero in 17246 500
updating nans in 18306 700 NOE
fixing zero in 19189 600
updating nans in 19356 600 NOE
updating nans in 36171 800 NOE


In [18]:
# List all the fields for each
tmp = df.groupby('entry_ID')['field_strength'].apply(list)

for x in tmp.index:
    if len(tmp[x])>1:
        print(x, ','.join(["%d"% y for y in tmp[x]]))
        
for x in tmp.index:
    print(x, ','.join(["%d"% y for y in sort(tmp[x])]))

4267 600,750,500
5330 500,600
5518 500,600,750
5687 500,600
5841 500,600
6838 500,600,800
15445 500,600
16392 500,600
19127 500,750
19153 600,900
27011 600,850,950
27194 500,700
27888 600,850
50233 600,750
51230 600,750
51416 600,800
4267 500,600,750
4390 500
4689 500
4762 750
5154 500
5330 500,600
5518 500,600,750
5687 500,600
5762 800
5839 600
5841 500,600
5991 500
5995 600
6470 600
6758 600
6838 500,600,800
7035 800
7036 800
7056 600
7432 600
11080 500
15255 600
15445 500,600
15451 500
15521 500
15795 500
16234 600
16392 500,600
16426 600
16737 600
17018 500
17246 500
17266 600
17306 600
17701 600
17783 600
17881 600
18092 600
18257 500
18260 600
18306 700
18380 600
18477 600
18758 400
18772 600
18773 600
18903 500
19127 500,750
19153 600,900
19189 600
19335 600
19356 600
25013 800
25636 600
26513 600
26723 600
27011 600,850,950
27194 500,700
27888 600,850
28132 850
30523 600
34545 700
36171 800
50020 500
50233 600,750
50332 600
50410 700
50734 500
51087 850
51230 600,750
51416 600,